### 1. How to use defaultdict?

In [3]:
from collections import defaultdict

dd = defaultdict(int)

dd['a', 1, 'cin'] = [1, 2, 3]
print(dd['a', 1, 'cin'])
print(dd[4])

[1, 2, 3]
0


### 2. How to use '**kwargs'?

In [2]:
def example_function(a, b, **kwargs):
    print('a:', a)
    print('b:', b)
    
    for key, value in kwargs.items():
        print(f'{key}={value}')
        
example_function(1, 2, c=3, d=4, e=5)

a: 1
b: 2
c=3
d=4
e=5


### 3. How to use 'wandb'?

In [1]:
import wandb
wandb.login(key = '7c6ed2a395454916ed21fc74401ae27278fba01c')

WANDB_PROJECT = "mnist-viz"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/shuogudaojin/.netrc


In [53]:
import torch 
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F 
import torch.utils.data.dataloader as DataLoader
import torchvision
import torchvision.transforms as T 


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# create train and test dataloaders
def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    ds = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    loader = torch.utils.data.DataLoader(dataset=ds, 
                                         batch_size=batch_size, 
                                         shuffle=True if is_train else False, 
                                         pin_memory=True, 
                                         num_workers=2)
    return loader

In [54]:
# Number of epochs to run
# Each epoch includes a training step and a test step, so this sets
# the number of tables of test predictions to log
EPOCHS = 1

# Number of batches to log from the test data for each test step
# (default set low to simplify demo)
NUM_BATCHES_TO_LOG = 10 #79

# Number of images to log per test batch
# (default set low to simplify demo)
NUM_IMAGES_PER_BATCH = 32 #128

# training configuration and hyperparameters
NUM_CLASSES = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001
L1_SIZE = 32
L2_SIZE = 64
# changing this may require changing the shape of adjacent layers
CONV_KERNEL_SIZE = 5

In [67]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.Layer1 = nn.Sequential(
            nn.Conv2d(1, L1_SIZE, CONV_KERNEL_SIZE, stride=1, padding=2),
            nn.BatchNorm2d(L1_SIZE),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.Layer2 = nn.Sequential(
            nn.Conv2d(L1_SIZE, L2_SIZE, CONV_KERNEL_SIZE, stride=1, padding=2),
            nn.BatchNorm2d(L2_SIZE),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*L2_SIZE, NUM_CLASSES)
        self.softmax = nn.Softmax(NUM_CLASSES)
    
    def forward(self, x):
        # uncomment to see the shape of a given layer:
        #print("x: ", x.size())
        out = self.Layer1(x)
        out = self.Layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [68]:
train_loader = get_dataloader(is_train=True, batch_size=BATCH_SIZE)
test_loader = get_dataloader(is_train=False, batch_size=2 * BATCH_SIZE)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [69]:
# ✨ W&B: Initialize a new run to track this model's training
wandb.init(project = "table-quickstart")
cfg = wandb.config
cfg.update({"epochs" : EPOCHS, "batch_size": BATCH_SIZE, "lr" : LEARNING_RATE,
            "l1_size" : L1_SIZE, "l2_size": L2_SIZE,
            "conv_kernel" : CONV_KERNEL_SIZE,
            "img_count" : min(10000, NUM_IMAGES_PER_BATCH*NUM_BATCHES_TO_LOG)})

# define model, loss, and optimizer
model = ConvNet(NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# convenience funtion to log predictions for a batch of test images
def log_test_predictions(images, labels, outputs, predicted, test_table, log_counter):
    # Obtain confidence scores for all classes
    scores = F.softmax(outputs, dim=1)
    log_scores = scores.cpu().numpy()
    log_images = scores.cpu().numpy()
    log_labels = labels.cpu().numpy()
    log_preds = predicted.cpu().numpy()
    # add ids based on the order of the images
    _id = 0
    for i, l, p, s in zip(log_images, log_labels, log_preds, log_scores):
        # add required info to data table:
        # id, image pixels, model's guess, true label, scores for all classes
        img_id = str(_id) + '_' +  str(log_counter)
        test_table.add_data(img_id, wandb.Image(i), p, l, *s)
        _id += 1
        if _id == NUM_IMAGES_PER_BATCH:
            break
        

In [71]:
# train the model
total_step = len(train_loader)
for epoch in range(EPOCHS):
    # training step
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # ✨ W&B: Log loss over training steps, visualized in the UI live
        wandb.log({'loss:': loss})
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss:{:.4f}'.format(epoch+1, EPOCHS, i+1, total_step, loss.item()))
            
    # ✨ W&B: Create a Table to store predictions for each test step
    columns = ['id', 'image', 'guess', 'truth']
    for digit in range(10):
        columns.append(f'score_' + str(digit))
    test_table = wandb.Table(columns=columns)
        
    # test the model
    model.eval()
    log_counter = 0
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            if log_counter < NUM_BATCHES_TO_LOG:
                log_test_predictions(images, labels, outputs, predicted, test_table, log_counter)
                log_counter += 1
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
                
        acc = 100 * correct / total
        # ✨ W&B: Log accuracy across training epochs, to visualize in the UI
        wandb.log({'epoch:': epoch, 'acc': acc})
        print('Test Accuraccy of the model on the 10000 test images:{}%'.format(acc))
            
    # ✨ W&B: Log predictions table to wandb
    wandb.log({'test_predictions': test_table})        
    
# ✨ W&B: Mark the run as complete (useful for multi-cell notebook)
wandb.finish()

Epoch [1/1], Step [100/1875], Loss:0.0022
Epoch [1/1], Step [200/1875], Loss:0.0282
Epoch [1/1], Step [300/1875], Loss:0.0313
Epoch [1/1], Step [400/1875], Loss:0.0611
Epoch [1/1], Step [500/1875], Loss:0.0553
Epoch [1/1], Step [600/1875], Loss:0.0045
Epoch [1/1], Step [700/1875], Loss:0.0292
Epoch [1/1], Step [800/1875], Loss:0.0085
Epoch [1/1], Step [900/1875], Loss:0.3448
Epoch [1/1], Step [1000/1875], Loss:0.0009
Epoch [1/1], Step [1100/1875], Loss:0.0007
Epoch [1/1], Step [1200/1875], Loss:0.0393
Epoch [1/1], Step [1300/1875], Loss:0.0096
Epoch [1/1], Step [1400/1875], Loss:0.0296
Epoch [1/1], Step [1500/1875], Loss:0.0131
Epoch [1/1], Step [1600/1875], Loss:0.0335
Epoch [1/1], Step [1700/1875], Loss:0.0469
Epoch [1/1], Step [1800/1875], Loss:0.0008


ValueError: Un-supported shape for image conversion [10]

In [1]:
import numpy as np
import torch
import torch.nn as nn
import math
from data_loader import get_dataset
from running import one_round_training
from methods import local_update
from models import CifarCNN, CNN_FMNIST, FC_SYN, ResNet50
from options import args_parser
import copy
from data_loader_synthetic import get_dataset_synthetic
from data_loader_medical import get_dataset_medical
import wandb
from svd_tools import initialize_grad_len
import torchvision.models as torch_models
torch.set_num_threads(4)

In [6]:
args = args_parser()
if 'synthetic' in args.dataset:
    train_loader, test_loader, global_train_loader, global_test_loader = get_dataset_synthetic(args)
elif args.dataset == 'retina' or args.dataset == 'covid_fl':
    train_loader, test_loader, global_train_loader, global_test_loader = get_dataset_medical(args)
else:
    train_loader, test_loader, global_train_loader, global_test_loader = get_dataset(args)

usage: ipykernel_launcher.py [-h] [--epochs EPOCHS] [--num_users NUM_USERS]
                             [--frac FRAC] [--local_epoch LOCAL_EPOCH]
                             [--local_iter LOCAL_ITER] [--local_bs LOCAL_BS]
                             [--lr LR] [--momentum MOMENTUM]
                             [--train_rule TRAIN_RULE] [--agg_g AGG_G]
                             [--lam LAM] [--local_size LOCAL_SIZE]
                             [--dataset DATASET] [--num_classes NUM_CLASSES]
                             [--device DEVICE] [--optimizer OPTIMIZER]
                             [--iid IID] [--noniid_s NONIID_S]
                             [--retina_split RETINA_SPLIT]
                             [--margin_loss_penalty MARGIN_LOSS_PENALTY]
                             [--k_proportion K_PROPORTION]
ipykernel_launcher.py: error: argument --frac: invalid float value: '/Users/shuogudaojin/Library/Jupyter/runtime/kernel-v2-40771n0mbHYTNuZ1.json'


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [3]:
from tqdm import tqdm

for i in tqdm(range(10000)):
    # Some computations
    pass

100%|██████████| 10000/10000 [00:00<00:00, 4193046.09it/s]


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import math
import json
from data_loader import get_dataset
from running import one_round_training
from methods import local_update
from models import CifarCNN, CNN_FMNIST, FC_SYN, ResNet50
from options import args_parser
import copy
from data_loader_synthetic import get_dataset_synthetic
from data_loader_medical import get_dataset_medical
import wandb
from svd_tools import initialize_grad_len
import torchvision.models as torch_models
torch.set_num_threads(4)



In [1]:
25 % 5

0